In [1]:
import pandas as pd
import sqlalchemy
import sys
import urllib

In [2]:
#Import shared functions
sys.path.append('..\..')
from IPM_Shared_Code_public.Python.utils import get_config

In [3]:
config = get_config('c:\Projects\config.ini')

driver = config['srv']['driver']
pip_server = config['srv']['pip']
pip = config['db']['pip']

In [4]:
con_string = 'Driver={' + driver + '};Server=' + pip_server +';Database=' + pip + ';Trusted_Connection=Yes;'
params = urllib.parse.quote_plus(con_string)
engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

In [8]:
sql = '''select table_name, 
                lower(column_name) as column_name, 
                null as column_description, 
                --ordinal_position, 
                lower(is_nullable) as is_nullable, 
                case when data_type in('nvarchar', 'varchar') then concat(data_type, '(', character_maximum_length, ')')
                     when data_type in('numeric', 'decimal')  then concat(data_type, '(', numeric_precision, ',', isnull(numeric_scale, ''), ')')
                     else data_type
                end as data_type
        from accessnewpip.information_schema.columns
        where table_name in('vw_pip_sync', 'allsites') '''

In [9]:
tables = pd.read_sql(con = engine, sql = sql)

In [10]:
print(tables.to_markdown(index = False))

| table_name   | column_name          | column_description   | is_nullable   | data_type     |
|:-------------|:---------------------|:---------------------|:--------------|:--------------|
| ALLSITES     | objectid             |                      | yes           | bigint        |
| ALLSITES     | propnum              |                      | yes           | nvarchar(25)  |
| ALLSITES     | prop id              |                      | no            | nvarchar(15)  |
| ALLSITES     | boro                 |                      | yes           | nvarchar(1)   |
| ALLSITES     | district             |                      | yes           | nvarchar(25)  |
| ALLSITES     | ampsdistrict         |                      | yes           | nvarchar(25)  |
| ALLSITES     | prop name            |                      | yes           | nvarchar(128) |
| ALLSITES     | site name            |                      | yes           | nvarchar(100) |
| ALLSITES     | prop location        |           